In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Load the dataset
data = pd.read_csv('xydata_2021winter.csv')

# Define target and features
target_columns_clusters = [f"Cluster{i}" for i in range(6)]  # Target variables
columns_to_drop = target_columns_clusters + ["Date", "Total_Accidents"]  # Non-predictive columns

# Retain only the most relevant features for X
selected_features = [
    "Time_Period", "Visibility (km)", "Temp (°C)", "Dew Point Temp (°C)",
    "Rel Hum (%)", "Wind Dir (10s deg)", "Wind Spd (km/h)", "Stn Press (kPa)",
    "Day_Of_Week", "Is_Weekend",  # Top-performing CxD-xHA features
    "C4D-1HA", "C5D-1HA", "C3D-1HA", "C4D-2HA", "C5D-2HA", "C3D-3HA"
]

# Define target and features
y = data[target_columns_clusters]  # Binary target
X = data[selected_features]  # Selected features

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("Training and testing split done.")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training and testing split done.
Training set: 338 samples
Testing set: 146 samples


In [3]:
# Step 1: Initialize Pipeline
# Define Random Forest within a pipeline
pipe_rf = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

print("Pipeline initialized.")

Pipeline initialized.


In [4]:
# Step 2: Define Parameter Grid
# Parameter grid for Random Forest
param_grid_rf = {
    'classifier__estimator__n_estimators': [100, 200, 500],
    'classifier__estimator__max_depth': [None, 10, 20],
    'classifier__estimator__min_samples_split': [2, 5],
}

In [5]:
# Step 3: Perform Grid Search with Cross-Validation
grid_rf = GridSearchCV(
    pipe_rf, param_grid_rf, cv=5, scoring='f1_weighted', return_train_score=True, verbose=1
)
grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('classifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))]),
             param_grid={'classifier__estimator__max_depth': [None, 10, 20],
                         'classifier__estimator__min_samples_split': [2, 5],
                         'classifier__estimator__n_estimators': [100, 200,
                                                                 500]},
             return_train_score=True, scoring='f1_weighted', verbose=1)

In [6]:
# Step 4: Print the Best Parameters
print("Best parameters:\n", grid_rf.best_params_)

# Best training and validation scores
best_train_score = max(grid_rf.cv_results_['mean_train_score'])
print(f"Best cross-validation train score: {best_train_score:.2f}")
best_val_score = grid_rf.best_score_
print(f"Best cross-validation validation score: {best_val_score:.2f}")

# Step 5: Evaluate on Test Set
y_test_pred = grid_rf.best_estimator_.predict(X_test)

# Test set F1-score
test_f1_score = f1_score(y_test, y_test_pred, average='weighted')
print(f"Test-set F1-score: {test_f1_score:.2f}")

# Step 6: Detailed Evaluation
print("Classification report on the test set:\n")
print(classification_report(y_test, y_test_pred, zero_division=0))

Best parameters:
 {'classifier__estimator__max_depth': None, 'classifier__estimator__min_samples_split': 5, 'classifier__estimator__n_estimators': 200}
Best cross-validation train score: 1.00
Best cross-validation validation score: 0.60
Test-set F1-score: 0.64
Classification report on the test set:

              precision    recall  f1-score   support

           0       0.57      0.42      0.48        55
           1       0.60      0.38      0.46        48
           2       0.69      0.65      0.67        75
           3       0.65      0.58      0.61        76
           4       0.74      0.71      0.73        73
           5       0.72      0.76      0.74        93

   micro avg       0.68      0.61      0.64       420
   macro avg       0.66      0.58      0.62       420
weighted avg       0.67      0.61      0.64       420
 samples avg       0.50      0.49      0.46       420

